# OCHL generator

## import functions

In [3]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=5153cc5b87a84c1c5b2bc6492286da8d0904a4a8c45c1086bcb8b706d3982cf7
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [1]:
!pip install plotly==4.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 7.7 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


In [4]:
from scipy.stats import skewnorm
import plotly.graph_objects as go
import plotly.express as px
import pandas_ta as ta
import pandas as pd
import numpy as np
import random

#normal distribution for difference between low and high
def create_pdf(sd, mean, alfa):
    #invertire il segno di alfa
    x = skewnorm.rvs(alfa, size=1000000)
    print(x)
    def calc(k, sd, mean):
        return (k*sd)+mean
    x = calc(x, sd, mean) #standard distribution
    return x

def graph_stock(df_original):
  #time_column is a string
  df = df_original.copy()
  pd.options.plotting.backend = "plotly"
  #i need to create a column with the index, otherwise plotly won't graph it
  df['date'] = df.index
  #fig = px.line(df, x='time', y=['price', 'price_2'])
  fig = px.line(df, x='date', y=df.columns)
  fig.show()
  #save to png
  #fig.write_image("csvfiles/btc.mark2mkt.png")

def graph_OCHL(df_OCHL, title):
    #fig_1 = px.line(df, x=df.index, y=df.columns, title=title)
    fig_2 = go.Figure(data=go.Ohlc(x=df_OCHL.index,
                        open=df_OCHL['open'],
                        high=df_OCHL['high'],
                        low=df_OCHL['low'],
                        close=df_OCHL['close']))
    fig_2.update(layout_xaxis_rangeslider_visible=False)
    fig_2.show()

def OCHL(group_values):
    min_ = min(group_values)
    max_ = max(group_values)
    range = max_ - min_
    open = min_+range*random.random()
    close = min_+range*random.random()
    return min_, max_, open, close

def simulate_stock(initial_price, drift, volatility, trend, days):
    days=days*24
    def create_pdf(sd, mean, alfa):
        #invertire il segno di alfa
        x = skewnorm.rvs(alfa, size=1000000)
        def calc(k, sd, mean):
            return (k*sd)+mean
        x = calc(x, sd, mean) #standard distribution
        return x

    def create_empty_df(days):
        #creare un empty DataFrame con le date
        empty = pd.DatetimeIndex(
            pd.date_range("2020-01-01", periods=days, freq="D")
        )
        empty = pd.DataFrame(empty)
        #si tagliano ore, minuti, secondi
        empty

        #si tagliano ore, minuti, secondi
        empty.index = [str(x)[0:empty.shape[0]] for x in list(empty.pop(0))]
        empty

        #final dataset con values
        stock = pd.DataFrame([x for x in range(0, empty.shape[0])])
        stock.index = empty.index
        return stock

    #ran = create_pdf(0.1, 0.2, 0) #del?
    #np.random.choice(ran) #del?
    #skeleton
    stock = create_empty_df(days)
    #initial price
    stock[0][0] = initial_price
    #create entire stock DataFrame
    x = create_pdf(volatility, drift, trend)
    for _ in range(1, stock.shape[0]):
        stock.iloc[_] = stock.iloc[_-1]*(1+np.random.choice(x))
    stock.index = pd.DatetimeIndex(stock.index)
    return stock

def simulate_OCHL_stock(df, graph_timeseries=False, graph_OCHL=False):
    df_ = list()
    #df.groupby(np.arange(len(df))//24).apply(OCHL) non funziona
    #sarebbe il modo corretto, ma devo creare un nuovo df da 0
    for a, b in df.groupby(np.arange(len(df))//24):
        group_values = np.array(b.values).flatten()
        low, high, open, close = OCHL(group_values)
        df_.append([low, high, open, close])

    #
    df_OCHL = pd.DataFrame(df_, index=pd.Series(pd.date_range("2020-01-01", periods=365, freq="D")), columns=['low', 'high', 'open', 'close'])

    #graph

    if graph_timeseries==True: graph_stock(df)
    if graph_OCHL==True:
        fig = go.Figure(
            data=go.Ohlc(x=df_OCHL.index,
            open=df_OCHL['open'],
            high=df_OCHL['high'],
            low=df_OCHL['low'],
            close=df_OCHL['close'])
        )
        fig.update(layout_xaxis_rangeslider_visible=False)
        fig.show()
    return df_OCHL

## generate stock

In [5]:
df_OCHL = simulate_OCHL_stock(simulate_stock(1000, 0, 0.01, 0, 365), graph_timeseries=True, graph_OCHL=True)
df_OCHL

,low,high,open,close
2020-01-01,985.337512,1037.451815,994.506427,1034.226032
2020-01-02,972.492009,1010.015995,978.584402,975.837483
2020-01-03,981.053474,1072.514937,1010.784212,1032.734223
2020-01-04,983.880950,1125.692020,1091.230335,1085.915777
2020-01-05,940.196586,1006.359648,972.247133,946.485646
...,...,...,...,...
2020-12-26,583.170701,612.951178,602.564677,602.809082
2020-12-27,578.790013,604.112897,602.589758,595.699766
2020-12-28,573.611671,604.016862,590.382582,592.205742
2020-12-29,562.250250,602.632135,569.601838,602.440010


## create backup

In [6]:
df_backup = df_OCHL.copy()

## strategy_1

In [8]:
#import backup
df = df_backup.copy()

MyStrategy = ta.Strategy(
    name="DCSMA10",
    ta=[
        {"kind": "ohlc4"},
        {"kind": "sma", "length": 10},
        {"kind": "sma", "length": 20},
    ]
)

# (2) Run the Strategy
df.ta.strategy(MyStrategy)
df = df.drop(['low', 'high', 'open'], axis=1)
graph_stock(df)

## strategy_2

In [9]:
#import backup
df = df_backup.copy()

MyStrategy = ta.Strategy(
    name="strategy_3",
    ta=[{
        "close": 'close',
        "kind": "bbands",
        "length": 15,
        "std": 2
    }]
)

# (2) Run the Strategy
df.ta.strategy(MyStrategy)
df = df.drop(['low', 'high', 'open'], axis=1)
graph_stock(df)